# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_path="../output_data/weather_data.csv"
weather_data_df=pd.read_csv(weather_data_path,encoding="utf-8")
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        576 non-null    object 
 1   lng         576 non-null    float64
 2   lat         576 non-null    float64
 3   Max Temp    576 non-null    float64
 4   Humidity    576 non-null    int64  
 5   Wind Speed  576 non-null    float64
 6   Cloudiness  576 non-null    int64  
 7   Country     572 non-null    object 
 8   Date        576 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 40.6+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store latitude and longitude in locations
locations = weather_data_df[["lat", "lng"]]

# Fill NaN values and convert to float
humidity = weather_data_df["Humidity"]

In [7]:
# Plot Heatmap
fig = gmaps.figure(center=(20.0, 80.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                max_intensity=100,
                                 dissipating=True,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# A max temperature between 15 to 30.
# Wind speed less than 15 mph.
# Humidity in between 25-50
# cloudiness in beween 10-35
Ideal_weather=weather_data_df.loc[(weather_data_df["Max Temp"]>15)&(weather_data_df["Max Temp"]<30)
                                  &(weather_data_df["Wind Speed"]<15)
                                  &(weather_data_df["Humidity"]>20)&(weather_data_df["Humidity"]<50)
                                  &(weather_data_df["Cloudiness"]>10)&(weather_data_df["Cloudiness"]<35)]
Ideal_weather.dropna(inplace = False) 
Ideal_weather.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 121 to 524
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        6 non-null      object 
 1   lng         6 non-null      float64
 2   lat         6 non-null      float64
 3   Max Temp    6 non-null      float64
 4   Humidity    6 non-null      int64  
 5   Wind Speed  6 non-null      float64
 6   Cloudiness  6 non-null      int64  
 7   Country     6 non-null      object 
 8   Date        6 non-null      int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 480.0+ bytes


In [9]:
Ideal_weather.head()

,city,lng,lat,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
121,Kruisfontein,24.7314,-34.0033,19.38,45,1.80,15,ZA,1624010632
357,Caluquembe,14.6833,-13.7833,22.45,25,2.85,21,AO,1624010695
422,Batetskiy,30.3027,58.6461,26.67,37,4.53,11,RU,1624010714
469,Nchelenge,28.7340,-9.3451,24.92,34,3.69,33,ZM,1624010726
496,Mount Darwin,31.5838,-16.7725,25.76,29,2.75,12,ZW,1624010735


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df=Ideal_weather[["Country","city","lng","lat"]]
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 121 to 524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  6 non-null      object 
 1   city     6 non-null      object 
 2   lng      6 non-null      float64
 3   lat      6 non-null      float64
dtypes: float64(2), object(2)
memory usage: 240.0+ bytes


In [11]:
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df['Address'] = ""



C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Find a hotel based on city_coords.
target_search = "Hotels"
radius = 5000
city_coords = hotel_df[["lng","lat"]]

# redefine params
params = {
    "location": city_coords,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["lat"]
    lng=row["lng"]
    city=row["city"]
    # get city location from df
    city_coords = f"{lat}, {lng}"
    print (city_coords)
    # add keyword to params dict
    params['location'] = city_coords

    # assemble url and make API request
   
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Address'] = results[0]['vicinity']
        print(f"Closest hotel is {results[0]['name']} for {city} .")
        print(f"Closest hotel {results[0]['name']} address is {results[0]['vicinity']} .")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


-34.0033, 24.7314
Missing field/result... skipping.
-13.7833, 14.6833


C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Hotel Kalukembe for Caluquembe .
Closest hotel Hotel Kalukembe address is Unnamed Road, Kalukembe .
58.6461, 30.3027
Missing field/result... skipping.
-9.3451, 28.734
Closest hotel is Relaxin Executive Lodge for Nchelenge .
Closest hotel Relaxin Executive Lodge address is Kashikishi .
-16.7725, 31.5838
Missing field/result... skipping.
36.9758, 120.7136
Closest hotel is Jin Jiang Golden Mountain International Hotel for Laiyang .
Closest hotel Jin Jiang Golden Mountain International Hotel address is No. 83 Heshan Road, Yantai .


In [13]:
hotel_df.head(8)

,Country,city,lng,lat,Hotel Name,Address
121,ZA,Kruisfontein,24.7314,-34.0033,,
357,AO,Caluquembe,14.6833,-13.7833,Hotel Kalukembe,"Unnamed Road, Kalukembe"
422,RU,Batetskiy,30.3027,58.6461,,
469,ZM,Nchelenge,28.7340,-9.3451,Relaxin Executive Lodge,Kashikishi
496,ZW,Mount Darwin,31.5838,-16.7725,,
524,CN,Laiyang,120.7136,36.9758,Jin Jiang Golden Mountain International Hotel,"No. 83 Heshan Road, Yantai"


In [14]:
# hotel_df
# nan_value = float("NaN")

hotel_df=hotel_df.replace("", np.nan, inplace=False)
# hotel_df_clear= hotel_df.dropna(subset=['Address'])
# filter = hotel_df['Address'] != ""
# hotel_df_clean = df[filter]
hotel_df.head(8)

,Country,city,lng,lat,Hotel Name,Address
121,ZA,Kruisfontein,24.7314,-34.0033,NaN,NaN
357,AO,Caluquembe,14.6833,-13.7833,Hotel Kalukembe,"Unnamed Road, Kalukembe"
422,RU,Batetskiy,30.3027,58.6461,NaN,NaN
469,ZM,Nchelenge,28.7340,-9.3451,Relaxin Executive Lodge,Kashikishi
496,ZW,Mount Darwin,31.5838,-16.7725,NaN,NaN
524,CN,Laiyang,120.7136,36.9758,Jin Jiang Golden Mountain International Hotel,"No. 83 Heshan Road, Yantai"


In [15]:
hotel_df_clean=hotel_df.dropna(inplace=False)
hotel_df_clean

,Country,city,lng,lat,Hotel Name,Address
357,AO,Caluquembe,14.6833,-13.7833,Hotel Kalukembe,"Unnamed Road, Kalukembe"
469,ZM,Nchelenge,28.7340,-9.3451,Relaxin Executive Lodge,Kashikishi
524,CN,Laiyang,120.7136,36.9758,Jin Jiang Golden Mountain International Hotel,"No. 83 Heshan Road, Yantai"


In [16]:
addresses=hotel_df_clean['Address'].tolist()
addresses

['Unnamed Road, Kalukembe', 'Kashikishi', 'No. 83 Heshan Road, Yantai']

In [19]:

# Create a map using city coordinates to set markers

# Create a marker_layer using the hotel info to fill the info box

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Hotel Adress</dt><dd>{Address}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
marker_locations = hotel_df_clean[['lat', 'lng']]
fig = gmaps.figure(center=(20.0, 80.0), zoom_level=1.5)
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]